# points ! code is in progress

In [ ]:
import numpy as np
import torch 
import cv2 as cv
import os 
import matplotlib.pyplot as plt 
from torchinfo import summary 
import torch.nn as nn
from typing import Tuple , Callable , Optional
from tqdm import tqdm

In [ ]:
# from typing import Literal 

# def capture_vide_frames(video_path:str  , video_name:str, 
#                         number_of_frames:int  , class_label:Literal[0, 1] =0 ): 

#     # capturing frames 
#     video_path = os.path.join(video_path , video_name)
#     cap = cv.VideoCapture(video_path)
    
#     total_frames = cap.get(cv.CAP_PROP_FRAME_COUNT)
#     frame_indices = np.random.uniform(1, total_frames , number_of_frames).astype('int')
#     count_frames = 0 
#     img_sample = []

#     while cap : 
#         isTrue, frame = cap.read()

#         if not isTrue : 
#             break

#         count_frames+=1 

#         if count_frames in frame_indices: 
#             img_sample.append(frame)

#         if cv.waitKey(10) & 0xff == ord('q'): 
#             break
        
#     cap.release()
#     cv.destroyAllWindows()

#     #optimizing the frames 
#     resized_images = [cv.resize(img, (224, 224)) for img in img_sample]

#     #saving the frames 
#     for idx,  img in enumerate(resized_images): 

#         if class_label : 
#             path = os.path.join('model_data/fire', video_name)
#             os.makedirs(path, exist_ok=True)
#             fileName= f'{idx}.jpg'
#             cv.imwrite(os.path.join(path, fileName) , img)

#         else : 
#             path = os.path.join('model_data/no_fire', video_name)
#             os.makedirs(path , exist_ok=True)
#             fileName = f'{idx}.jpg'
#             Image.open(os.path.join(path, fileName) , img)


In [ ]:
root_dir = '/media/ai/datasets/firesmoke_dataset'

In [ ]:
fire_vidoes_list  = os.listdir(os.path.join(root_dir , 'fire' ))
no_fire_video_list  = os.listdir(os.path.join(root_dir , 'no_fire'))

len(fire_vidoes_list) , len(os.listdir(os.path.join(root_dir,  'fire', 'firesmoke_2')))


In [ ]:
# for vid in fire_vidoes_list: 
#     capture_vide_frames('data/videos/fire', vid, 40, 1)


# for vid in no_fire_video_list:
#     capture_vide_frames('data/videos/nofire' , vid , 100)

In [ ]:
# import glob 

# fire_images = glob.glob('model_data/fire/**/*')
# no_fire_images = glob.glob('model_data/no_fire/**/*')

# print(len(fire_images), len(no_fire_images))

In [ ]:
from torch.utils.data import Dataset, DataLoader , random_split
from PIL import Image
from torchvision import transforms 

In [ ]:
class FireVideoDataset(Dataset):
    def __init__(self, root_dir, chunk_size=30):
        self.chunk_size = chunk_size
        self.samples = []
        self.transform = transforms.Compose([
            transforms.ToTensor(),  # Convert PIL image to min max scale (0-1)
            transforms.Normalize(
                    [0.485, 0.456, 0.406],
                    [0.229, 0.224, 0.225]
                    ), 
            transforms.Resize((224, 224))
            # transforms.RandomHorizontalFlip(0.5), 
            # transforms.ColorJitter(0.2)
        ])
        for class_name, label in [('fire', 1), ('no_fire', 0)]:
            class_dir = os.path.join(root_dir, class_name)
            for chunk_folder in os.listdir(class_dir):
                chunk_path = os.path.join(class_dir, chunk_folder)
                
                if os.path.isdir(chunk_path) and len(os.listdir(chunk_path)) >= chunk_size:
                    self.samples.append((chunk_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        chunk_path, label = self.samples[idx]
        
        image_files = os.listdir(chunk_path)[:self.chunk_size] 
        frames = []

        for fname in image_files:
            img_path = os.path.join(chunk_path, fname)
            img = Image.open(img_path)

            if img is None:
                continue  
            frames.append(self.transform(img)) 
        np_frame = np.array(frames)  # its [B , C , H , W]
        np_frame = np.transpose(np_frame, (1 , 0, 2, 3))  # becomes [C , B , H , W]

        return torch.tensor(np_frame, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)


In [ ]:
dataset = FireVideoDataset(root_dir=root_dir)

train_size = int(0.7 *len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_data_loader = DataLoader(train_dataset, batch_size=2,shuffle=True , num_workers=8)
test_data_loader = DataLoader(test_dataset, batch_size=2, shuffle=True , num_workers=8)

# for batch_idx, (data, label) in enumerate(train_data_loader):
#     if batch_idx == 2:
#         break
#     print(data.shape)

In [ ]:
from torch.utils.data import Subset, random_split, DataLoader
import random

# Subset 10% of the dataset
subset_size = int(len(dataset) * 0.1)

# Get a random subset of the dataset (10%)
subset_data, _ = random_split(dataset, [subset_size, len(dataset) - subset_size])

# Train/test split sizes (as integers)
train_subset_size = int(subset_size * 0.8)
test_subset_size = subset_size - train_subset_size

# Split the subset into train and test
subset_train, subset_test = random_split(subset_data, [train_subset_size, test_subset_size])

# Create dataloaders
check_data_loader = DataLoader(subset_train, batch_size=2, shuffle=True, num_workers=8, drop_last=True)
test_check_data_loader = DataLoader(subset_test, batch_size=2, shuffle=True, num_workers=8, drop_last=True)

# Check sizes
print(len(check_data_loader), len(test_check_data_loader))


In [ ]:
# pos = 0 
# neg = 0 
# for i , label in check_data_loader: 
#     if 1 in label: 
#         pos += 1 
#     else : 
#         neg += 1
        
# print(pos, neg)

In [ ]:

class FireDetector(nn.Module):
    def __init__(self, in_channels=3, num_classes=1):
        super().__init__()
        self.conv_block = nn.Sequential(
            nn.Conv3d(in_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(),
            nn.MaxPool3d((1, 2, 2)),  # frame , hight , width (keep fram , max pool each frame)

            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(),
            nn.MaxPool3d((2, 2, 2)), # pick two frame , maxpool each frame 

            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool3d((1, 1, 1))  # compress to (1,1,1) # pick one frame , one pixel(best one in hight and ..)
        )
        self.classifier = nn.Linear(64, num_classes) # [9, 64  ,  1 ,1 , 1] we resize to have channels giving to linear to pick 2 

    def forward(self, x):
        x = self.conv_block(x)
        x = x.squeeze() # flatten
        x = self.classifier(x)
        return x


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
# fire_detector_model = FireDetector()
# summary(fire_detector_model)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
def measure_acc(ypred , ytrue): 
    print(ytrue.shape )
    from torchmetrics import Accuracy , Recall
    acc_fn = Accuracy(task='binary').to(device)
    acc = acc_fn(ypred , ytrue)

    sig_ypred = (torch.sigmoid(ypred) > 0.5).int()
    recallfn = Recall(task='binary').to(device)
    recall = recallfn(sig_ypred , ytrue)
    return acc , recall

In [ ]:

def train_and_eval_model(model,
                         train_loader: DataLoader,
                         val_loader: DataLoader,
                         loss_fn: torch.nn.Module,
                         number_of_epochs: int, 
                        #  pack_path_way:Optional[Callable[[int , int], torch.tensor]]
                         ) -> Tuple[list, list, list, list]:

    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, weight_decay=1e-3)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    train_recall , val_recalls = [] , []

    for epoch in range(number_of_epochs):

        # -------- TRAINING -------- #

        model.train()
        running_loss, running_acc, running_recall = 0.0, 0.0, 0.0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{number_of_epochs} - Training"):
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            y_pred = y_pred.squeeze()
            loss = loss_fn(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            acc, recall = measure_acc(y_pred, y)
            running_recall += recall.item()
            running_acc += acc.item()
        epoch_train_recall = running_recall/len(train_loader)
        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = running_acc / len(train_loader)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)
        train_recall.append(epoch_train_recall)

        # -------- EVALUATION -------- #

        model.eval()
        val_loss, val_acc , val_recall= 0.0, 0.0 , 0.0
        with torch.inference_mode():
            for x, y in tqdm(val_loader, desc=f"Epoch {epoch+1}/{number_of_epochs} - Validation"):
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                y_pred = y_pred.squeeze()
                loss = loss_fn(y_pred, y)

                val_loss += loss.item()
                acc, recall = measure_acc(y_pred, y)
                val_recall += recall.item()
                val_acc += acc.item()

        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = val_acc / len(val_loader)
        epoch_val_recall = val_recall/len(val_loader)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)
        val_recalls.append(epoch_val_recall)


        print(f"[Epoch {epoch+1}] Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} | Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    return train_losses, train_accuracies , train_recall, val_losses, val_accuracies , val_recall


In [ ]:
import numpy
print(numpy.__version__)

In [ ]:
# train_loss , train_acc , recall= train_and_eval_model(fire_detector_model , train_data_loader , test_data_loader, loss_fn , 1)
# print(f'train loss is {train_loss:2f}\n',
#        f'train accuracy is {train_acc:.2f}\n', 
#        f'recall is {recall:.2f}\n')

In [ ]:
# test_loss , test_acc , recall = eval_the_model(fire_detector_model  , test_data_loader , loss_fn)
# print(f'train loss is {train_loss:2f}\n',
#        f'train accuracy is {train_acc:.2f}\n', 
#        f'recall is {recall:.2f}\n')


In [ ]:
# from torchvision.models.video import r3d_18 , R3D_18_Weights
# resnet_model = r3d_18(weights=R3D_18_Weights.DEFAULT)
# resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 1)  # Binary classification

# resnet_model.fc

In [ ]:
# train_loss , train_acc , recall= train_and_eval_model(resnet_model , 1, train_data_loader , loss_fn  )
# print(f'train loss is {train_loss:2f}\n',
#        f'train accuracy is {train_acc:.2f}\n', 
#        f'recall is {recall:.2f}\n')


In [ ]:
# test_loss , test_acc , recall = eval_the_model(resnet_model ,  test_data_loader, loss_fn)
# print(f'test loss is {test_loss}\n',
#        f'test accuracy is {test_acc}\n', 
#        f'recall is {recall}\n')

### trying conv2dand1D

In [ ]:
class FireDetectorConv2D1D(nn.Module): 
    def __init__(self, in_channels = 3 , num_classes=1): 
        super().__init__() 
        self.spatial_extractor = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size = 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(), 
            nn.MaxPool2d(2),

            nn.Conv2d(32 , 64, kernel_size = 3 , padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
        )

        self.temporal_extractor = nn.Sequential(
            nn.Conv1d(64, 128 , 3 ,padding = 1 ),
            nn.BatchNorm1d(128), 
            nn.ReLU(), 
            nn.AdaptiveAvgPool1d(1) 
        )

        self.clasifier = nn.Linear(128, num_classes)

    def forward(self , x): 
            B , C , T , H , W = x.size()
            x = x.permute(0, 2, 1, 3 ,4)  # (B, T, C, H, W)
            x = x.reshape(B * T , C , H , W)
            x = self.spatial_extractor(x) # (B*T, 64, 1, 1)
            x = x.view(B , T , 64)  # (B, T, 64)

            x = x.permute(0 , 2 , 1) # (B , 64, T)
            x = self.temporal_extractor(x) # (B , 128, 1)
            x = x.squeeze(-1) # (B , 128)

            return self.clasifier(x)

In [ ]:
model2d_plus = FireDetectorConv2D1D()
summary(model2d_plus)

In [ ]:
train_losses, train_accuracies , train_recall, val_losses, val_accuracies , val_recalls = train_and_eval_model(FireDetectorConv2D1D(), check_data_loader, test_check_data_loader, loss_fn, 1)

In [ ]:
print(
       f'train loss is {train_losses[-1]:2f}\n',
       f'train accuracy is {train_accuracies[-1]:.2f}\n', 
       f'recall is {train_recall[-1]:.2f}\n'
       f'val loss is {val_losses[-1]:2f}\n',
       f'val accuracy is {val_accuracies[-1]:.2f}\n', 
       # f'val recall is {val_recalls[-1]:.2f}\n'
       )

In [ ]:
# test_loss , test_acc , recall = eval_the_model(model2d_plus, test_data_loader, loss_fn )
# print(f'test loss is {test_loss:2f}\n',
#        f'test accuracy is {test_acc:.2f}\n', 
#        f'recall is {recall:.2f}\n')

In [ ]:
class FireDetectorMain3D(nn.Module): 
    def __init__(self, in_channels , out_channels, kernel_size, padding): 
        super().__init__()
        T, H, W = kernel_size

        self.spatial_conv = nn.Conv3d(
            in_channels,  out_channels , kernel_size=(1 , H, W), padding=(0, padding , padding),
        )

        self.relu = nn.ReLU()
        
        self.temporal_conv = nn.Conv3d(
            out_channels , out_channels , kernel_size=(T , 1, 1) , padding=(padding, 0 , 0)
        )
    
    def forward(self, x): 
        x = self.spatial_conv(x)
        x = self.relu(x) 
        x = self.temporal_conv(x) 
        return x 

class Residual(nn.Module): 
    def __init__(self, channels , kernel_size, padding =1):
        super().__init__()
        self.conv1 = FireDetectorMain3D(channels, channels, kernel_size , padding)
        self.norm1 = nn.LayerNorm(channels)
        self.conv2 = FireDetectorMain3D(channels ,channels, kernel_size , padding)
        self.norm2 = nn.LayerNorm(channels)
        self.relu = nn.ReLU()

    def forward(self, x) : 
        Residual = x 
        out = self.conv1(x) #(N, C, D, H, W)
        out = out.permute(0, 2 , 3 , 4 , 1)
        out = self.norm1(out)
        out= out.permute(0 , 4 , 1 , 2, 3)
        # out = self.relu(out)

        out = self.conv2(out)
        out = out.permute(0, 2 , 3 , 4 , 1)
        out = self.norm2(out)
        out= out.permute(0 , 4 , 1 , 2, 3)

        out += Residual
        out = self.relu(out)
        return out 
    
class FireDetectorWithResidual(nn.Module): 
        def __init__(self , in_channels =3 , num_clases =1) :
            super().__init__()
            self.initial_conv = FireDetectorMain3D(in_channels, 16 , kernel_size=(3, 7, 7) , padding=1)
            self.bn = nn.BatchNorm3d(16)
            self.relu = nn.ReLU() 
            self.pool1 = nn.MaxPool3d((1 , 2 ,2))
            self.res_block1 = Residual(16, kernel_size=(3 ,3 ,3))
            self.pool2 = nn.MaxPool3d((2 ,2 ,2))

            self.res_block2 = Residual(16, kernel_size=(3 ,3 ,3))
            self.adaptive_pool = nn.AdaptiveAvgPool3d((1 ,1,1))

            self.classifier = nn.Linear(16 , num_clases)

        def forward(self  , x): 
            x = self.initial_conv(x)
            x = self.bn(x)
            x = self.relu(x)
            x = self.pool1(x)

            x = self.res_block1(x)
            x = self.pool2(x)

            x = self.res_block2(x)
            x = self.adaptive_pool(x)

            x = x.flatten(1)
            x = self.classifier(x)

            return x

In [ ]:
# test_sample = next(iter(train_data_loader))[0]

In [ ]:

resnet2d1d = FireDetectorWithResidual()
# summary(resnet2d1d, input_size=test_sample.shape)  # (batch, C, D, H, W)


In [ ]:
train_losses, train_accuracies , train_recall, val_losses, val_accuracies , val_recalls  = train_and_eval_model(resnet2d1d, check_data_loader, test_check_data_loader, loss_fn, 1)

In [ ]:
print(
       f'train loss is {train_losses[-1]:2f}\n',
       f'train accuracy is {train_accuracies[-1]:.2f}\n', 
       f'recall is {train_recall[-1]:.2f}\n'
       f'val loss is {val_losses[-1]:2f}\n',
       f'val accuracy is {val_accuracies[-1]:.2f}\n', 
       # f'val recall is {val_recalls[-1]:.2f}\n'
       )

In [ ]:
# test_loss, test_acc , test_recall = eval_the_model(resnet2d1d , test_data_loader, loss_fn)
# print(f'test loss is {test_loss}\n',
#        f'test accuracy is {test_acc}\n', 
#        f'recall is {recall}\n')

## slow fast model

In [ ]:
import torch
# Choose the `slowfast_r50` model 
slow_fast_model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50' , pretrained=True)

In [ ]:
summary(slow_fast_model)

In [ ]:

# from torchvision.transforms import Compose, Lambda, CenterCrop, Normalize 

# from torchvision.transforms.v2 import (
#     CenterCropVideo,
#     NormalizeVideo,
# )
from torchvision.transforms import v2 

In [ ]:
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
)
from torchvision.transforms import Compose


In [ ]:
slow_fast_model = slow_fast_model.eval()
slow_fast_model = slow_fast_model.to(device)

In [ ]:
from torchvision.transforms import Compose, Lambda, CenterCrop, Normalize

In [ ]:
# video = EncodedVideo.from_path('data/videos/fire/fire_2.mp4')
# clip = video.get_clip(start_sec=0, end_sec=1.0)  # Extract a 2-second segment
# clip['video'].shape

In [ ]:
# transform = ApplyTransformToKey(
#     key="video",
#     transform=Compose([
#         # UniformTemporalSubsample(32), # its for videos , samples are ready
#         Lambda(lambda x: x / 255.0),
#         NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
#         ShortSideScale(256),
#         CenterCropVideo(224),
#     ])
# )

In [ ]:
# clip['video'].shape

In [ ]:
# myclip = {'video' :  clip['video']}
# myclip['video'].shape

In [ ]:
# ts_image = transform(myclip)
# final_ts_image = ts_image['video'].unsqueeze(0)
# final_ts_image.shape

In [ ]:
def pack_pathway_output(frames, alpha=4):
    fast_pathway = frames
    slow_pathway = frames[:, :, ::alpha, :, :]  # temporal subsampling
    return [slow_pathway, fast_pathway]


In [ ]:
test = cv.imread('image.png')
test = test.reshape(1, 1, 768, 768, 3)
test.shape

In [ ]:
test = test.transpose(0, 4 , 1, 2, 3)
test.shape

In [ ]:
import torch
import numpy as np

# Example image (H, W, C)
img = np.random.rand(768, 768, 3).astype(np.float32)

# Convert to torch tensor and permute to (C, H, W)
img_tensor = torch.from_numpy(img).permute(2, 0, 1)  # Now (3, 768, 768)

# Add time and batch dimensions → (1, 3, 1, 768, 768)
img_tensor_5d = img_tensor.unsqueeze(0).unsqueeze(2)

print(img_tensor_5d.shape)  # Output: torch.Size([1, 3, 1, 768, 768])


In [ ]:
import numpy as np

img = np.random.rand(3, 768, 768)  # (C, H, W)

# Repeat frame 20 times along depth (time) axis
depth = 32
video = np.repeat(img[np.newaxis, ...], depth, axis=0)  # Shape: (D, C, H, W)

# Rearrange to (C, D, H, W)
video = np.transpose(video, (1, 0, 2, 3))  # Shape: (C, D, H, W)

# Add batch dimension: (1, C, D, H, W)
video = np.expand_dims(video, axis=0)

print(video.shape)  # Output: (1, 3, 20, 768, 768)


In [ ]:
# video = torch.from_numpy(video)
# video.shape

In [ ]:
video = video.float()

In [ ]:
video = video.squeeze()
video.shape

In [ ]:
ob_img = {'video' : video}

In [ ]:
ob_img['video'].shape


In [ ]:
# torch.Size([2, 3, 30, 224, 224])

In [ ]:
ts_img = transform(ob_img)

In [ ]:
ts_img['video'].shape

In [ ]:
final_ts_img = ts_img['video'].unsqueeze(0)
final_ts_img.shape

In [ ]:
ready = pack_pathway_output(final_ts_img)
ready[0].shape , ready[1].shape

In [ ]:
with torch.inference_mode(): 
    slow_fast_model.to('cpu') 
    output = slow_fast_model(ready)
    output

In [ ]:
output.shape

In [ ]:
torch.argmax(output , dim=1).shape

In [ ]:
predicted_class = torch.softmax(output, dim=1)
predicted_class.shape

In [ ]:
torch.argmax(predicted_class)

In [ ]:
def slow_fast_model_train(model, train_loader:DataLoader , val_loader:DataLoader, 
                          loss_fn,
                          number_of_epochs:int ,
                          pathway_alpha:int)-> Tuple[list , list , list , list]:

    def pack_pathway_output(frames, alpha):
        fast_pathway = frames
        slow_pathway = frames[:, :, ::alpha, :, :]  # temporal subsampling
        return [slow_pathway, fast_pathway]
    
    def padd_to_32(data:torch.tensor) : 
            T = data.shape[2]
            if T == 32 : 
                return data 
            elif T < 32: 
                padd = 32 - T 
                st1 =  data[:,:,-1:,:,:].repeat(1, 1, padd , 1, 1)
                stacked = torch.cat([st1, data] , dim=2)
                return stacked
            else: 
                return data[:, :, :32, : , :]
            

    def transform_data(data:torch.tensor): 

        # transform = ApplyTransformToKey(
        # key="video",
        # transform=Compose([
        #     # UniformTemporalSubsample(32), # its for videos , samples are ready
        #     Lambda(lambda x: x / 255.0),
        #     NormalizeVideo(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225]),
        #     ShortSideScale(256),
        #     CenterCropVideo(224),
        # ]))
        
        #input shape N, C , D , H , W 
        # p_data = data.permute(0, 2 , 1, 3 ,4)
        # N, D, C, H, W = p_data.shape
        # r_data = p_data.reshape(N * D, C , H, W)
        # r_data = {'video' : r_data}
        # t_data = transform(r_data)['video']
        # data = t_data.reshape(N, D , C , H , W)    
        # data = data.permute(0 , 2 , 1, 3, 4)  

        data = pack_pathway_output(data , pathway_alpha)   # this transofrms frame to slow and fast double frames 
        
        return data
        
        
        
        ## in progrsss

    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3, weight_decay=1e-3)

            
    optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    train_recalls , val_recalls = [] , []

    for epoch in range(number_of_epochs):
        running_loss , running_acc , running_recall = 0 , 0 , 0 
        for x , y in tqdm(train_loader , desc=f'Epoch {epoch+1}/{number_of_epochs} - Training' ): 
            # ----- TRAINING ----- #
            x , y = x.to(device) , y.to(device)
            x = padd_to_32(x)
            x = transform_data(x)

            model.train() 
            ypred = model(x).squeeze()
            ypred = torch.argmax(ypred ,  dim=1)
            print(ypred.shape)
            loss = loss_fn(ypred, y)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            acc , recall = measure_acc(ypred, y)
            running_loss += loss.item() 
            running_acc  += acc 
            running_recall += recall
        epoch_train_recall = running_recall/len(train_loader)
        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = running_acc / len(train_loader)
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)
        train_recalls.append(epoch_train_recall)        

    # ----- EVALUATION ---- # 

        model.eval()
        val_loss, val_acc , val_recall= 0.0, 0.0 , 0.0
        with torch.inference_mode():
            for x, y in tqdm(val_loader, desc=f"Epoch {epoch+1}/{number_of_epochs} - Validation"):
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                y_pred = y_pred.squeeze()
                loss = loss_fn(y_pred, y)

                val_loss += loss.item()
                acc, recall = measure_acc(y_pred, y)
                val_recall += recall.item()
                val_acc += acc.item()

        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = val_acc / len(val_loader)
        epoch_val_recall = val_recall/len(val_loader)
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)
        val_recalls.append(epoch_val_recall)

        print(f"[Epoch {epoch+1}] Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f} | Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    return train_losses, train_accuracies , train_recall, val_losses, val_accuracies , val_recall




In [ ]:
train_losses, train_accuracies ,train_recall, val_losses,val_accuracies , val_recall  = slow_fast_model_train(slow_fast_model , check_data_loader, test_check_data_loader , loss_fn, 1  , 4)
